In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Balance

In [ ]:
from portfolio.balance import BalanceType
b = Balance()
print("Currencies",  b.currencies)
print("Entries", b.to_dict())
b.add_currency(c.ETH)
print("Currencies",  b.currencies)
b.update(c.ETH, delta_free=1.0, delta_used=0.0)
print(c.ETH, b.get(c.ETH)[BalanceType.FREE])

### Exchange

In [ ]:
# Public informaiton
exchanges = [c.PAPER, c.BINANCE, c.GDAX]#, c.POLONIEX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    exchange.get_markets()
    exchange.fetch_ohlcv(a, Timeframe.ONE_MIN.value['id'])
    exchange.fetch_order_book(a)
    exchange.fetch_public_trades(a)
    exchange.fetch_ticker(a)

In [ ]:
# Account Information
exchanges = [c.PAPER, c.BINANCE, c.GDAX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.fetch_balance().to_dict())
    print(exchange.fetch_orders(a))
    print(exchange.fetch_open_orders(a))
    print(exchange.fetch_closed_orders(a))

In [ ]:
exchange = load_exchange(c.BINANCE)
a = Asset(c.BTS, c.BTC)

# Market BUY ... CAREFUL THIS WILL ACTUALLY BUY
print("Exchange", exchange)
#order_dict = exchange.create_market_buy_order(a, 1)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
print(order_dict)

In [ ]:
# Check Order
balance = exchange.fetch_balance()
print(balance.to_dict())
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
found_order = exchange.fetch_order(order_dict['id'], a.symbol)
print("Placed order", found_order) 

In [ ]:
# Sell all remaining quantity
print(a.base)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base)[BalanceType.TOTAL])
order = exchange.create_market_sell_order(a, balance.get(a.base)[BalanceType.TOTAL])
print(order)

In [ ]:
print(exchange.fetch_balance()[a.base], exchange.fetch_balance()[a.quote])

### Data

In [ ]:
base = c.LTC
quote = c.USDT
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.BTC, c.ETH, c.LTC] ]
period = '1m' 
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [ ]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period, start, end)
df.head()

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period)
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period, start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period, start)
df.head()

### Data Feed

In [ ]:
# CSV Feed
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)
feed.initialize()

In [ ]:
# Grab 1 row at a time
for i in range(3):
    data = feed.next()
    print(data['time_utc'], data['close'])

In [ ]:
# Access all rows in history
feed.history().head()

In [ ]:
# Exchange Live Feed
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
end = None
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')

In [ ]:
feed = ExchangeDataFeed(exchange, [assets[0]], period, feed_fpath, start, end)
feed.initialize()

In [ ]:
# Access History
feed.history(t_minus=3).tail()

In [ ]:
# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = feed.next(refresh=False)
    print(data['time_utc'], data['close'])

In [ ]:
# Refresh data before next query
data = feed.next(refresh=True)
print(data['time_utc'], data['close'])

In [ ]:
# Example Using the Feed

def my_strategy(row, history, exchange):
    print("Time:", row['time_utc'], "Price:", row['close'])
    if row['close'] > .024:
        print("Buying coin")
        #res = exchange.limit_buy(quantity=1, price=5)
    else:
        print("Selling that stupid coin")
    return {
        'time': row['time_utc'],
        'close': row['close'],
        'action': 'buy',
        'quantity': 1,
        'price': 5,
        'exchange': exchange.id
    }

def backtest(feed, strategy, exchange):
    results = []
    row = feed.next()
    while row is not None:
        output = strategy(row, feed, exchange)
        row = feed.next(refresh=True)
        results.append(output)
    return results

In [ ]:
res = backtest(feed, my_strategy, exchange)

### Order

In [ ]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

### Position

In [ ]:
asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1, cost_price=250.0)
print("Asset", pos.asset.to_dict())
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(-1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Go short
pos.update(-2, 150)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Close the short for a LOSS and go long (price went up, we had to pay $300 to cover our short)
# then we bought another share because we're newbs
pos.update(2, 300)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

In [ ]:
"""
Assume that an investor made the following consecutive fund purchases in a taxable account: 1,500 shares at $20, 1,000 shares at $10 and 1,250 shares at $8. The investor’s average cost basis is calculated by dividing $50,000/3,750 shares. The average cost is $13.33.

Suppose the investor then sells 1,000 shares of the fund at $19.

Gain/loss using average cost basis: ($19 - $13.33) x 1,000 shares = $5,667
"""

asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1500, cost_price=20.0)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1000, txn_price=10)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1250, txn_price=8)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

last_cost = pos.cost_price
pos.update(txn_quantity=-1000, txn_price=19)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
print("Profit", (19 - last_cost) * 1000)

### PerformanceTracker

In [ ]:
asset = Asset(c.BTC, c.USD)
perf = PerformanceTracker(starting_cash=5000, timeframe=Timeframe.ONE_MIN, store=None)
perf.to_dict()

In [ ]:
# Buy 1 BTC for $1000
pos1 = Position(asset, quantity=1, cost_price=1000.0)
positions = [pos1]
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased $100
Position Return
    Return = .10
    PnL = $100
Cumulative Return
    Return = $100 / $5000 = .02
    PnL = $100
"""
pos1.latest_price = 1100
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased again $400
Position Return
    Return = .5
    PnL = $500
Cumulative Return
    Return = $500 / $5000 = .1
    PnL = $500
"""
pos1.latest_price = 1500
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
# Buy 1 ETH for $500
asset = Asset(c.ETH, c.USD)
pos2 = Position(asset, quantity=1, cost_price=500.0)
positions.append(pos2)
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of ETH decreased $100
Position Return
    Return = -100/500
    PnL = -100
Cumulative Return
    Return = 400 / 5000 = .08
    PnL = 400
"""
pos2.latest_price = 400
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

### Portfolio

In [ ]:
starting_cash = 5000
exchange = load_exchange(c.PAPER)
perf = PerformanceTracker(starting_cash, Timeframe.ONE_MIN, store=None)
portfolio = Portfolio(starting_cash, perf)

In [ ]:
# Buy 1 BTC for $1000
asset = Asset(c.BTC, c.USD)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1000., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

In [ ]:
portfolio.update(filled_orders=[order])
portfolio.to_dict()

In [ ]:
# Buy more BTC, price has risen $500
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
portfolio.update(filled_orders=[order])
portfolio.to_dict()

In [ ]:
# Sell all BTC at profit
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=2, 
    order_type=OrderType.LIMIT_SELL
)
portfolio.update(filled_orders=[order])
portfolio.to_dict()

In [ ]:
portfolio.perf.to_dict()

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics

class Strategy():
    def __init__(self, params):
        self.params = params
        self.initialize()
    
    def initialize(self):
        self.ctx.symbol = 'BTC/USD'
    
    def handle_data(self, data):
        # Handle next iteration of data from feed
        pass
    
class TestStrategy():
    def __init__(self):
        pass
    
    def next(self, data):
        pass

### Plotting

In [ ]:
utils.charts.plot_range(feed.history(), start, end, 'close')

### Runner

In [ ]:
@unique
class TradeMode(Enum):
    BACKTEST = 0
    SIMULATE = 1
    LIVE = 2


class Punisher():
    def __init__(self, exchange, feed, strategy, record):
        self.exchange = exchange
        self.feed = feed
        self.strategy = strategy
        self.record = record
        
    def backtest(self):
        results = []
        row = self.feed.next()
        while row is not None:
            print("Timestep", row['time_utc'], "Price", row['close'])
            row = self.feed.next()
            output = self.strategy(row, self.exchange, self.feed)
            results.append(output)
        return results
    
    def simulate(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['test'].append(output)
            time.sleep(2)

    def live(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['live'].append(output)
            time.sleep(2)
    
    def punish(self, mode=False):
        if mode == TradeMode.BACKTEST:
            print("Backtesting ...")
            self.backtest()
        elif mode == TradeMode.SIMULATE:
            print("Simulating orders ...")
            self.simulate()
        elif mode == TradeMode.LIVE:
            print("LIVE TRADING! CAREFUL!")
            self.live()

# PaperExchangeDataProvider

In [ ]:

base = c.LTC
quote = c.USDT
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.BTC, c.ETH, c.LTC] ]
period = '1m' 
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)

data_provider = PaperExchangeDataProvider(feed)
config = {
    'data_provider': data_provider,
    'balance': Balance()
}

exchange = load_exchange("paper", config)


In [ ]:
# Testing Paper Exchange Data Provider
print(exchange.get_markets())
print(exchange.fetch_ohlcv(asset, '1m'))
print(exchange.fetch_order_book(asset))
print("TICKER")
print(exchange.fetch_ticker(asset))
print(exchange.timeframes)
print(exchange.fetch_public_trades(asset))

In [10]:
context = Context.from_config()
punisher.punish(context, SimpleStrategy())

Loading feed: /Users/cfortuner/Dropbox/punisher/default_feed.csv
Downloading: ETH/BTC
Downloaded rows: 500
Punishing ...
Timestep 2018-01-09 17:54:00 Price 0.08082
2018-01-10T02:30:51.409207, Epoch 1
2018-01-10T02:30:51.409309, open                    0.08076
high                    0.08095
low                    0.080759
close                  0.080824
volume                  178.429
time_utc    2018-01-09 17:55:00
Name: 1515520500, dtype: object


AttributeError: 'NoneType' object has no attribute 'to_csv'